In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
latest_data = pd.read_csv("./Latest reported counts of COVID-19 cases and death.csv")
weekly_data = pd.read_csv("./Weekly COVID-19 cases and deaths by date reported to WHO.csv")
daily_data = pd.read_csv("./WHO-COVID-19-global-daily-data.csv")

# daily_data 

In [ ]:
daily_data.head(100)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-04,SB,Solomon Islands,WPR,0.0,0,0.0,0
1,2020-01-04,LK,Sri Lanka,SEAR,0.0,0,0.0,0
2,2020-01-04,SY,Syrian Arab Republic,EMR,NaN,0,NaN,0
3,2020-01-04,TJ,Tajikistan,EUR,NaN,0,NaN,0
4,2020-01-04,TH,Thailand,SEAR,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...
95,2020-01-04,AF,Afghanistan,EMR,NaN,0,NaN,0
96,2020-01-04,AD,Andorra,EUR,NaN,0,NaN,0
97,2020-01-04,AR,Argentina,AMR,NaN,0,NaN,0
98,2020-01-04,BJ,Benin,AFR,NaN,0,NaN,0


In [3]:
daily_data["Date_reported"] = pd.to_datetime(daily_data.Date_reported)

In [ ]:
print(daily_data.Date_reported.min())
print(daily_data.Date_reported.max())

2020-01-04 00:00:00
2024-12-22 00:00:00


In [4]:
# correct number of weeks between start and end date
#  daily_data.groupby("Date_reported").count()
daily_data.Date_reported.nunique()

1815

In [ ]:
daily_data.isna().sum()

Date_reported             0
Country_code           1815
Country                   0
WHO_region                0
New_cases            227896
Cumulative_cases          0
New_deaths           226960
Cumulative_deaths         0
dtype: int64

In [ ]:
print('no.unique country: ', daily_data["Country"].nunique())
print('no. unique country code: ', daily_data["Country_code"].nunique())

no.unique country:  240
no. unique country code:  239


In [4]:
daily_data[daily_data["Country_code"].isna()]['Country'].unique()

array(['Namibia'], dtype=object)

In [ ]:
country = []
nan_cases = []
nan_deaths = []
max_cases = []
max_deaths = []

form = {
    'country':country,
    'nan_cases':nan_cases,
    'nan_deaths':nan_deaths,
    'max_cases':max_cases,
    'max_deaths':max_deaths
}

for c in daily_data['Country'].unique():
    df = daily_data[daily_data['Country'].eq(c)]
    
    country.append(c)
    nan_cases.append(df.isna().sum()['New_cases'])
    nan_deaths.append(df.isna().sum()['New_deaths'])
    max_cases.append(df["Cumulative_cases"].max())
    max_deaths.append(df["Cumulative_deaths"].max())
    


pd.DataFrame(form)

,country,nan_cases,nan_deaths,max_cases,max_deaths
0,Solomon Islands,387,387,25954,199
1,Sri Lanka,340,338,672812,16907
2,Syrian Arab Republic,878,876,57423,3163
3,Tajikistan,1388,1388,17786,125
4,Thailand,330,330,4808454,34741
...,...,...,...,...,...
235,Mauritania,916,913,63879,997
236,Mexico,628,628,7622513,334818
237,Montenegro,1611,1611,251280,2654
238,New Caledonia,385,385,80203,314


In [55]:
daily_data[daily_data['New_cases'].isna() & daily_data['Country'].eq('Syrian Arab Republic')].head(10)

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
19202,2020-03-24,SY,Syrian Arab Republic,EMR,NaN,1,0.0,0
19649,2020-03-25,SY,Syrian Arab Republic,EMR,NaN,1,0.0,0
20130,2020-03-27,SY,Syrian Arab Republic,EMR,NaN,5,0.0,0
20162,2020-03-28,SY,Syrian Arab Republic,EMR,NaN,5,0.0,0
20609,2020-03-29,SY,Syrian Arab Republic,EMR,NaN,5,0.0,0
21122,2020-04-01,SY,Syrian Arab Republic,EMR,NaN,10,NaN,2
21569,2020-04-02,SY,Syrian Arab Republic,EMR,NaN,10,NaN,2
21602,2020-04-03,SY,Syrian Arab Republic,EMR,NaN,10,NaN,2
22082,2020-04-05,SY,Syrian Arab Republic,EMR,NaN,16,NaN,2
22562,2020-04-07,SY,Syrian Arab Republic,EMR,NaN,19,NaN,2


### dealing with missing values

In [ ]:
# fill missing value in country code
# nambia country code is NA
daily_data["Country_code"] = daily_data["Country_code"].fillna('NA')

* steps to deal with missing value in new cases and new deaths
* first step: if cummulative death or case == 0, fill new death and case with 0
* second step: check cummulative cases with new cases per country

In [8]:
# fill the missing value where cummulative counterpart = 0
daily_data.loc[(daily_data['New_cases'].isna()) & (daily_data['Cumulative_cases'].eq(0)), 'New_cases'] = 0
daily_data.loc[(daily_data['New_deaths'].isna()) & (daily_data['Cumulative_deaths'].eq(0)), 'New_deaths'] = 0

In [18]:
def fill_new_col(df, new_col, cumulative_col):
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Initialize the new column if it doesn't exist
    if new_col not in result_df.columns:
        result_df[new_col] = None
    
    # Process each country separately
    for country in result_df['Country'].unique():
        # Create a mask for the current country
        mask = result_df['Country'] == country
        
        # Get the country-specific data
        country_data = result_df.loc[mask]
        
        # Calculate differences and fill values
        result_df.loc[mask, new_col] = country_data[cumulative_col].diff()
        
        # Fill first value (which will be NaN after diff()) with the cumulative value
        first_idx = country_data.index[0]
        result_df.loc[first_idx, new_col] = result_df.loc[first_idx, cumulative_col]
    
    return result_df


daily_data2 = fill_new_col(daily_data, new_col='New_cases', cumulative_col='Cumulative_cases')
daily_data2 = fill_new_col(daily_data2, new_col='New_deaths', cumulative_col='Cumulative_deaths')

In [19]:
daily_data2.isna().sum()

Date_reported        0
Country_code         0
Country              0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
dtype: int64

*  dim date [date reported, year, month, day, dayweek, ]
*  dim location [country, country code, who code]
*  fact [new cases, new deaths, cummulative cases, cummulative deaths] 

# latest_data

In [9]:
latest_data.head()

,Name,WHO Region,Cases - cumulative total,Cases - cumulative total per 100000 population,Cases - newly reported in last 7 days,Cases - newly reported in last 7 days per 100000 population,Cases - newly reported in last 24 hours,Deaths - cumulative total,Deaths - cumulative total per 100000 population,Deaths - newly reported in last 7 days,Deaths - newly reported in last 7 days per 100000 population,Deaths - newly reported in last 24 hours
0,Belarus,Europe,994045.0,10520.0,0.0,NaN,0.0,7118.0,75.0,0.0,NaN,0.0
1,China,Western Pacific,99381579.0,6755.0,56.0,NaN,56.0,122391.0,8.0,2.0,NaN,2.0
2,French Guiana,NaN,98041.0,32825.0,NaN,NaN,NaN,413.0,138.0,NaN,NaN,NaN
3,Latvia,Europe,977765.0,51254.0,0.0,NaN,0.0,7475.0,392.0,0.0,NaN,0.0
4,Saint Vincent and the Grenadines,Americas,9674.0,8720.0,NaN,NaN,NaN,124.0,112.0,NaN,NaN,NaN


# weekly_data

In [11]:
weekly_data.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-05,AF,Afghanistan,EMRO,NaN,0,NaN,0
1,2020-01-12,AF,Afghanistan,EMRO,NaN,0,NaN,0
2,2020-01-19,AF,Afghanistan,EMRO,NaN,0,NaN,0
3,2020-01-26,AF,Afghanistan,EMRO,NaN,0,NaN,0
4,2020-02-02,AF,Afghanistan,EMRO,NaN,0,NaN,0


In [16]:
weekly_data.isna().sum()

Date_reported            0
Country_code           261
Country                  0
WHO_region            4698
New_cases            16637
Cumulative_cases         0
New_deaths           16489
Cumulative_deaths        0
dtype: int64

In [25]:
weekly_data["Date_reported"] = pd.to_datetime(weekly_data.Date_reported)
print(weekly_data.Date_reported.min())
print(weekly_data.Date_reported.max())

2020-01-05 00:00:00
2024-12-29 00:00:00


In [26]:
# correct number of weeks between start and end date
weekly_data["Date_reported"].nunique()

261